In [4]:
import os
import sys
# Add parent directory to sys.path (this is your project root)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
import xarray as xr
import matplotlib.pyplot as plt
import torch
import pandas as pd
import seaborn as sns
import numpy as np
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
from esda.moran import Moran
from libpysal.weights import KNN
from eval.metrics import *
from eval.plot import *
import data.valid_crd as valid_crd

In [45]:
# Define start and end date
clim = 'access_cm2'
ref = 'gfdl_esm4'
start_date = "1981-01-01"
end_date = "1995-12-31"
period = [2075, 2099]
degree = 1
testep = 20
emph_quantile = 0.9
scenario = 'ssp5_8_5'

cmip6_dir = '/pscratch/sd/k/kas7897/cmip6'
ref_path = '/pscratch/sd/k/kas7897/cmip6/gfdl_esm4'
if clim ==  'ensemble':
    y_clim = 'access_cm2'
else:
    y_clim = clim

ds_sample = xr.open_dataset(f"{cmip6_dir}/{y_clim}/historical/precipitation//clipped_US.nc")
valid_coords = valid_crd.valid_lat_lon(ds_sample, var_name='pr')


y = torch.load(f'/pscratch/sd/k/kas7897/diffDownscale/jobs/{clim}-{ref}/QM_ANN_layers4_degree{degree}_quantile{emph_quantile}/all/1950_1980/{scenario}_{period[0]}_{period[1]}/y.pt', weights_only = False).to('cpu').squeeze(-1).numpy()
x = torch.load(f'/pscratch/sd/k/kas7897/diffDownscale/jobs/{clim}-{ref}/QM_ANN_layers4_degree{degree}_quantile{emph_quantile}/all/1950_1980/{scenario}_{period[0]}_{period[1]}/x.pt', weights_only = False).to('cpu').squeeze(-1).numpy()
time = torch.load(f'/pscratch/sd/k/kas7897/diffDownscale/jobs/{clim}-{ref}/QM_ANN_layers4_degree{degree}_quantile{emph_quantile}/all/1950_1980/{scenario}_{period[0]}_{period[1]}/time.pt', weights_only = False)
time_y = torch.load(f'/pscratch/sd/k/kas7897/diffDownscale/jobs/{clim}-{ref}/QM_ANN_layers4_degree{degree}_quantile{emph_quantile}/all/1950_1980/{scenario}_{period[0]}_{period[1]}/time_y.pt', weights_only = False)


xt = torch.load(f'/pscratch/sd/k/kas7897/diffDownscale/jobs/{clim}-{ref}/QM_ANN_layers4_degree{degree}_quantile{emph_quantile}/all/1950_1980/{scenario}_{period[0]}_{period[1]}/ep{testep}/xt.pt', weights_only = False)
# xt = xr.open_dataset(f'/pscratch/sd/k/kas7897/cmip6/ensemble/1950_1980/{period[0]}_{period[1]}/diffDownscale/precipitation/ensemble.nc')
# xt = xt['pr'].sel(lat=xr.DataArray(valid_coords[:, 0], dims='points'),
#                                     lon=xr.DataArray(valid_coords[:, 1], dims='points'),
#                                     method='nearest').values


loca = xr.open_dataset(f'{cmip6_dir}/{clim}/{scenario}/precipitation/loca/coarse_USclip.nc')
loca = loca['pr'].sel(lat=xr.DataArray(valid_coords[:, 0], dims='points'),
                                    lon=xr.DataArray(valid_coords[:, 1], dims='points'),
                                    method='nearest')
loca = loca.sel(time =slice(f'{period[0]}', f'{period[1]}')).values

#unit conversion
loca = loca*86400


QM_bench = f'/pscratch/sd/k/kas7897/diffDownscale/benchmark/QuantileMapping/conus/{clim}-{ref}/[1950, 1980]_{scenario}_{period}.pt'
QM_debiased = torch.load(QM_bench, weights_only=False).squeeze(-1)
QM_debiased = QM_debiased*86400

In [46]:
## this block filters 'y' based on 'x' calender

x_time_np = np.array([pd.Timestamp(str(t)) for t in time])
x_time_np = np.array([pd.Timestamp(t).replace(hour=0, minute=0, second=0) for t in x_time_np], dtype='datetime64[D]')

y_time_np = np.array([pd.Timestamp(str(t)) for t in time_y])
y_time_np = np.array([pd.Timestamp(t).replace(hour=0, minute=0, second=0) for t in y_time_np], dtype='datetime64[D]')


# Find commonindices where observed time matches model time
common_times, y_idx, x_idx = np.intersect1d(y_time_np, x_time_np, return_indices=True)

y = y[y_idx,:]
x = x[x_idx,:]
xt = xt[x_idx,:]
loca = loca[x_idx,:]
QM_debiased = QM_debiased[x_idx,:]

x_time_np = x_time_np[x_idx]
y_time_np = y_time_np[y_idx]

### Season Filtering

In [48]:
y_s = load_seasonal_data(y_time_np, y)
x_s = load_seasonal_data(x_time_np, x)
xt_s = load_seasonal_data(x_time_np, xt)

loca_s = load_seasonal_data(x_time_np, loca)
# QM_s = load_seasonal_data(x_time_np, QM_debiased)


t_s = load_seasonal_data(x_time_np, x_time_np)

In [2]:
# Initialize climate indices manager
climate_indices = ClimateIndices()

day_bias_percentages = get_day_bias_percentages(x, y, xt, climate_indices)
mean_bias_percentages = get_mean_bias_percentages(x, y, xt, x_time_np, climate_indices)

loca_day_bias_percentages = get_day_bias_percentages(x, y, loca, climate_indices)
loca_mean_bias_percentages = get_mean_bias_percentages(x, y, loca, x_time_np, climate_indices)


QM_day_bias_percentages = get_day_bias_percentages(x, y, QM_debiased, climate_indices)
QM_mean_bias_percentages = get_mean_bias_percentages(x, y, QM_debiased, x_time_np, climate_indices)

## Arranging delCLIMAD and LOCA in one dictionary
for key in day_bias_percentages.keys():
    day_bias_percentages[key] = day_bias_percentages[key] + (loca_day_bias_percentages[key][1],) + (QM_day_bias_percentages[key][1],)


for key in mean_bias_percentages.keys():
    mean_bias_percentages[key] = mean_bias_percentages[key] + (loca_mean_bias_percentages[key][1],) + (QM_mean_bias_percentages[key][1],)

NameError: name 'ClimateIndices' is not defined

In [ ]:
keys = ['SDII (Monthly)','CDD (Yearly)', 'CWD (Yearly)', "Rx1day", "Rx5day", "R10mm",  "R20mm", "R95pTOT", "R99pTOT"]
d = dict(filter(lambda item: item[0] in keys , mean_bias_percentages.items()))

keys = ["Dry Days", "Wet Days >1mm", "Very Wet Days >10mm", "Very Very Wet Days >20mm"]
d4 = dict(filter(lambda item: item[0] in keys , day_bias_percentages.items()))

# Create a 2x2 subplot figure
fig, axes = plt.subplots(2, 1, figsize=(24, 12), sharey=True)

# Ensure axes is flattened for easy indexing
axes = axes.flatten()
method_names = [f"delCLIMAD-BA(degree{degree}_quantile{emph_quantile}_ep{testep})", "LOCA"]


# Call the function for each dataset
plot_violin_bias(axes[0], d, "Bias(%)", "Mean Bias(%) for Different Precipitation Indices",  method_names=method_names, remove_outlier=True)

plot_violin_bias(axes[1], d4, "Bias(%)", "Day Bias(%) for Different Precipitation Indices",  method_names=method_names, remove_outlier=True)
fig.suptitle(f'{clim}', fontsize=20, fontweight="bold", y=1.02)


plt.tight_layout()
plt.show()